In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

2025-07-15 11:43:53.297199: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-15 11:43:53.333433: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-15 11:43:53.333457: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-15 11:43:53.334338: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-15 11:43:53.341578: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# I think we can get the metrics in a for loop for all the blocks individually?
# And later we will compute an average metric too

In [3]:
# get teh trained model?
non_overlapping_model = tf.keras.models.load_model("models/CNN_seq2seq_non_overlapping.keras")

2025-07-15 11:43:55.342176: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


In [4]:
# locate the test data

# input features
input_features_loc = 'data/test_input_sub_images'
input_contents = os.listdir(input_features_loc)
input_contents.sort()

In [5]:
# test targets
out_targets_loc = 'data/test_out_targets'
out_contents = os.listdir(out_targets_loc)
out_contents.sort()

In [6]:
# out_contents

In [7]:
%%time
# now how to proceed?

test_rmse = []
test_mae = []
test_r2 = []
test_personr = []
preds = []
for i in range(len(input_contents)):
    # load the features
    test_features = np.load(os.path.join(input_features_loc, input_contents[i]))
    # load targets
    test_targets = np.load(os.path.join(out_targets_loc, out_contents[i]))
    # predict the targets with trained model
    test_preds = non_overlapping_model.predict(test_features)
    preds.append(test_preds)
    # compute the test scores, I think we need to flatten these before computing the scores - or can use tf, but the answers are going to be the same
    test_preds_flatten = test_preds.flatten()
    test_targets_flatten = test_targets.flatten()
    mae = mean_absolute_error(test_targets_flatten, test_preds_flatten)
    test_mae.append(mae)
    rmse = np.sqrt(mean_squared_error(test_targets_flatten, test_preds_flatten))
    test_rmse.append(rmse)
    rsquare = r2_score(test_targets_flatten, test_preds_flatten)
    test_r2.append(rsquare)
    pearsonr_score = pearsonr(test_targets_flatten, test_preds_flatten)[0]
    test_personr.append(pearsonr_score)

2025-07-15 11:43:56.805395: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


29/29 [==============================] - 0s 8ms/step
CPU times: user 8.45 s, sys: 16.5 s, total: 24.9 s
Wall time: 8.52 s


In [8]:
# test_preds.shape

In [9]:
# preds[0].shape

In [10]:
# Make sure to also have a train script without relu activation

In [11]:
test_rmse

[0.30817848,
 0.29190674,
 0.30174834,
 0.28407466,
 0.3162589,
 0.30801052,
 0.28900272,
 0.34542254,
 0.31593293,
 0.2976515,
 0.29901716,
 0.3307212,
 0.31968552]

In [12]:
test_mae

[0.14700195,
 0.13444678,
 0.14118999,
 0.13030937,
 0.1495598,
 0.1457912,
 0.13270512,
 0.16239701,
 0.14796063,
 0.13996702,
 0.13976054,
 0.15674771,
 0.14942487]

In [13]:
test_r2

[0.7572155017253988,
 0.8072256836862943,
 0.7869350146691263,
 0.8120676055729109,
 0.7461398226447957,
 0.7327348127718495,
 0.8124007329928593,
 0.7019205179568582,
 0.7448710631458986,
 0.7610479280833753,
 0.7762393128828577,
 0.7214341103869577,
 0.7455742017969514]

In [14]:
test_personr

[0.8703177904540282,
 0.9001042966932388,
 0.8909525926670814,
 0.9032772176558256,
 0.8643709314151917,
 0.8560129503473228,
 0.9032438505503798,
 0.8378380784981865,
 0.8631710840815606,
 0.8728201193603392,
 0.8813033170444302,
 0.8494642538656734,
 0.8634747021326035]

In [15]:
# Now, do this for all test data at once?

In [16]:
# stack all input and outputs

In [17]:
all_test_features_list = [np.load(os.path.join(input_features_loc, file)) for file in input_contents]

In [18]:
# all_test_features_list[0].shape

In [19]:
# len(all_test_features_list)

In [20]:
# stack all data together
all_test_features = np.vstack(all_test_features_list)

In [21]:
all_test_features.shape

(11830, 13, 30, 30, 3)

In [22]:
all_test_targets_list = [np.load(os.path.join(out_targets_loc, file)) for file in out_contents]

In [23]:
# all_test_targets_list[0].shape

In [24]:
# len(all_test_targets_list)

In [25]:
# stack all data together
all_test_targets = np.vstack(all_test_targets_list)

In [26]:
all_test_targets.shape

(11830, 7, 32)

In [27]:
# get all test predictions
all_test_preds = non_overlapping_model.predict(all_test_features)

370/370 [==============================] - 3s 9ms/step


In [28]:
all_test_preds.shape

(11830, 7, 32)

In [29]:
# flatten these vectors
all_test_targets_flatten = all_test_targets.flatten()
all_test_preds_flatten = all_test_preds.flatten()

In [30]:
all_test_targets_flatten.shape, all_test_preds_flatten.shape

((2649920,), (2649920,))

In [31]:
# compute metrics
all_test_rmse = np.sqrt(mean_squared_error(all_test_targets_flatten, all_test_preds_flatten))
all_test_mae = mean_absolute_error(all_test_targets_flatten, all_test_preds_flatten)
all_test_r2_score = r2_score(all_test_targets_flatten, all_test_preds_flatten)
all_test_pearsonr = pearsonr(all_test_targets_flatten, all_test_preds_flatten)

In [32]:
all_test_rmse, all_test_mae, all_test_r2_score, all_test_pearsonr[0]

(0.3087269, 0.14440475, 0.7638292342329516, 0.8743625975982096)